 ## Stastics about the data

In [187]:
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import dask_ml
import joblib
import numpy as np
import pandas as pd 
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import time
import warnings
warnings.filterwarnings('ignore')
import random
from xgboost import XGBClassifier
# import shutil
import os
from our_evaluate_sepsis_score import evaluate_performance_dask
import our_functions_library as flib

In [198]:
path = "C:/Users/ZhuRui/OneDrive - KU Leuven/Master thesis sepsis 20-21/Our code/K-fold/raw_data"
# path = "C:/Users/ZhuRui/OneDrive - KU Leuven/Master thesis sepsis 20-21/Our code/K-fold/raw_data"

In [199]:
filename = 'raw_data_all.csv' # use raw dataset
dataFilePath = os.path.join(path,filename)
originalData = pd.read_csv(dataFilePath,dtype={'Gender':bool,'SepsisLabel':bool}) # read csv data into Dask DataFrame
Uniq_ID= np.unique(originalData['Patient_id'])
print("Data size:",originalData.shape)
print('Patient id size:',len(Uniq_ID))
print(Uniq_ID)
X_full_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_full_columns = ['Patient_id', 'SepsisLabel']

Data size: (1552210, 43)
Patient id size: 40336
[    1     2     3 ... 40334 40335 40336]


In [200]:
print("Original Data set size:",originalData.shape)
print(originalData.memory_usage(deep=True).sum(),"bytes")

Original Data set size: (1552210, 43)
512229428 bytes


In [216]:
missing = originalData.isna().sum()/len(originalData)*100
# print(len(originalData))
# print(missing)
# missing = missing/len(originalData)*100
# print("Missing value ratios: \n",missing)
# print(missing.sort_values(ascending=False))
type(missing)
missing.name = "full missing ratio"
print(missing)

HR                   9.882619
O2Sat               13.061119
Temp                66.162697
SBP                 14.576958
MAP                 12.451279
DBP                 31.345887
Resp                15.354559
EtCO2               96.286843
BaseExcess          94.579020
HCO3                95.810618
FiO2                91.665754
pH                  93.069688
PaCO2               94.440121
SaO2                96.549372
AST                 98.377604
BUN                 93.134434
Alkalinephos        98.393194
Calcium             94.116067
Chloride            95.460279
Creatinine          93.904433
Bilirubin_direct    99.807371
Glucose             82.894325
Lactate             97.329872
Magnesium           93.689578
Phosphate           95.986303
Potassium           90.689082
Bilirubin_total     98.509158
TroponinI           99.047745
Hct                 91.145979
Hgb                 92.617558
PTT                 97.055875
WBC                 93.593199
Fibrinogen          99.340167
Platelets 

In [202]:
healthyData = originalData[originalData['SepsisLabel'] == False]
sepsisData = originalData[originalData['SepsisLabel'] == True]

In [265]:
healthyMeans = healthyData.mean().round(2)
sepsisMeans = sepsisData.mean().round(2)
healthyStds = healthyData.std().round(2)
sepsisStds = sepsisData.std().round(2)
healthyMissing = round(healthyData.isna().sum()/len(healthyData)*100,3)
sepsisMissing = round(sepsisData.isna().sum()/len(sepsisData)*100,3)

In [266]:
healthyMeans.name = "healthy_means"
sepsisMeans.name = "sepsis_means"
healthyStds.name = "healthy_std"
sepsisStds.name = "sepsis_std"
healthyMissing.name = "healthy_missing"
sepsisMissing.name = "sepsis_missing"
dif_means = pd.Series((sepsisMeans - healthyMeans),name = "dif_means")
dif_ratio_abs = pd.Series(round(dif_means/healthyMeans*100,2).abs(),name = "dif_ratio_abs")

In [267]:
stasdf = pd.DataFrame(healthyMeans)
stasdf = stasdf.join(sepsisMeans)
stasdf = stasdf.join(healthyStds)
stasdf = stasdf.join(sepsisStds)
stasdf = stasdf.join(healthyMissing)
stasdf = stasdf.join(sepsisMissing)
stasdf = stasdf.join(dif_means)
stasdf = stasdf.join(dif_ratio_abs)

In [275]:
stasdf =stasdf.sort_values("dif_ratio_abs",ascending=False)

In [277]:
stasdf.to_csv("Full data analsis.csv")

,healthy_means,sepsis_means,healthy_std,sepsis_std,healthy_missing,sepsis_missing,dif_means,dif_ratio_abs
HR,84.47,90.79,17.27,19.01,9.918,7.942,6.32,7.48
O2Sat,97.20,96.99,2.93,3.26,13.111,10.360,-0.21,0.22
Temp,36.97,37.25,0.76,1.05,66.165,66.055,0.28,0.76
SBP,123.79,121.45,23.20,24.99,14.566,15.196,-2.34,1.89
MAP,82.44,80.17,16.33,16.63,12.505,9.496,-2.27,2.75
DBP,63.86,62.02,13.95,14.10,31.373,29.886,-1.84,2.88
Resp,18.69,20.46,5.07,6.14,15.373,14.343,1.77,9.47
EtCO2,32.99,32.43,7.97,7.65,96.415,89.264,-0.56,1.70
BaseExcess,-0.70,-0.35,4.27,4.96,94.667,89.769,0.35,50.00
HCO3,24.09,23.70,4.36,4.92,95.847,93.817,-0.39,1.62
